In [21]:
import numpy as np
import pandas as pd 
import sklearn
import scipy.sparse 

for p in [np, pd, sklearn, scipy]:
    print (p.__name__, p.__version__)

numpy 1.13.3
pandas 0.23.4
sklearn 0.19.1
scipy 0.19.1


In [22]:
train_path = 'X.npz'
train_labels = 'Y.npy'

test_path = 'X_test.npz'
test_labels = 'Y_test.npy'

# Train data
X = scipy.sparse.load_npz(train_path)
Y = np.load(train_labels)

# Test data
X_test = scipy.sparse.load_npz(test_path)
Y_test = np.load(test_labels)

# Out-of-fold features we loaded above were generated with n_splits=4 and skf seed 123
# So it is better to use seed 123 for generating KNN features as well 
skf_seed = 123
n_splits = 4

In [43]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors import NearestNeighbors
from multiprocessing import Pool

import numpy as np


class NearestNeighborsFeats(BaseEstimator, ClassifierMixin):
    '''
        This class should implement KNN features extraction 
    '''
    def __init__(self, n_jobs, k_list, metric, n_classes=None, n_neighbors=None, eps=1e-6):
        self.n_jobs = n_jobs
        self.k_list = k_list
        self.metric = metric
        
        if n_neighbors is None:
            self.n_neighbors = max(k_list) 
        else:
            self.n_neighbors = n_neighbors
            
        self.eps = eps        
        self.n_classes_ = n_classes
    
    def fit(self, X, y):
        '''
            Set's up the train set and self.NN object
        '''
        # Create a NearestNeighbors (NN) object. We will use it in `predict` function 
        self.NN = NearestNeighbors(n_neighbors=max(self.k_list), 
                                      metric=self.metric, 
                                      n_jobs=1, 
                                      algorithm='brute' if self.metric=='cosine' else 'auto')
        self.NN.fit(X)
        
        # Store labels 
        self.y_train = y
        
        # Save how many classes we have
        self.n_classes = np.unique(y).shape[0] if self.n_classes_ is None else self.n_classes_
        
        
    def predict(self, X):       
        '''
            Produces KNN features for every object of a dataset X
        '''
        if self.n_jobs == 1:
            test_feats = []
            for i in range(X.shape[0]):
                test_feats.append(self.get_features_for_one(X[i:i+1]))
        else:
            '''
                 *Make it parallel*
                     Number of threads should be controlled by `self.n_jobs`  
                     
                     
                     You can use whatever you want to do it
                     For Python 3 the simplest option would be to use 
                     `multiprocessing.Pool` (but don't use `multiprocessing.dummy.Pool` here)
                     You may try use `joblib` but you will most likely encounter an error, 
                     that you will need to google up (and eventually it will work slowly)
                     
                     For Python 2 I also suggest using `multiprocessing.Pool` 
                     You will need to use a hint from this blog 
                     http://qingkaikong.blogspot.ru/2016/12/python-parallel-method-in-class.html
                     I could not get `joblib` working at all for this code 
                     (but in general `joblib` is very convenient)
                     
            '''
            
            # YOUR CODE GOES HERE
            indice_chunks = np.array_split(range(X.shape[0]), self.n_jobs)
            pool = Pool(processes=self.n_jobs)
            
            def process_indice_chunk(indice_chunk):
                test_feats_ = []
                for i in indice_chunks:
                    test_feats_.append(self.get_features_for_one(X[i:i+1]))
                return test_feats_
                
            job_results = pool.map(process_indice_chunk, indice_chunks)
            for job_result in job_results:
                test_feats.append(job_result)
            # test_feats =  # YOUR CODE GOES HERE
            # YOUR CODE GOES HERE
            
            #assert False, 'You need to implement it for n_jobs > 1'
            
            
            
        return np.vstack(test_feats)
        
        
    def get_features_for_one(self, x):
        '''
            Computes KNN features for a single object `x`
        '''

        NN_output = self.NN.kneighbors(x)
        
        # Vector of size `n_neighbors`
        # Stores indices of the neighbors
        neighs = NN_output[1][0]
        
        # Vector of size `n_neighbors`
        # Stores distances to corresponding neighbors
        neighs_dist = NN_output[0][0] 

        # Vector of size `n_neighbors`
        # Stores labels of corresponding neighbors
        neighs_y = self.y_train[neighs] 
        
        ## ========================================== ##
        ##              YOUR CODE BELOW
        ## ========================================== ##
        
        # We will accumulate the computed features here
        # Eventually it will be a list of lists or np.arrays
        # and we will use np.hstack to concatenate those
        return_list = [] 
        
        
        ''' 
            1. Fraction of objects of every class.
               It is basically a KNNСlassifiers predictions.

               Take a look at `np.bincount` function, it can be very helpful
               Note that the values should sum up to one
        '''
        
        labels = np.unique(self.y_train)
        #print("labels", labels)

        for k in self.k_list:
            # YOUR CODE GOES HERE
            k_nearest = neighs_y[0:k]
            binned = np.bincount(k_nearest)
            feats = np.zeros(self.n_classes)
            for i,b in enumerate(binned):
                feats[i] = b/k
                
            #print(feats)
            #print("n_classes", self.n_classes)
            #print("k", k)
            #print("neighs_y", neighs_y)
            #print("neighs_dist", neighs_dist)
            
            assert len(feats) == self.n_classes
            return_list += [feats]
            
        #print("1:", return_list)
        
        '''
            2. Same label streak: the largest number N, 
               such that N nearest neighbors have the same label.
               
               What can help you: `np.where`
        '''
        N = 1
        for label in np.unique(neighs_y):
            indices = np.where(np.array(neighs_y) == label)
            consecutive = np.array(range(np.min(indices), np.max(indices)+1))
            if np.array_equal(indices, consecutive):
                length = len(consecutive)
                if length > N:
                    N = length
        feats = [N]# YOUR CODE GOES HERE
        
        assert len(feats) == 1
        return_list += [feats]
        
        '''
            3. Minimum distance to objects of each class
               Find the first instance of a class and take its distance as features.
               
               If there are no neighboring objects of some classes, 
               Then set distance to that class to be 999.

               `np.where` might be helpful
        '''
        feats = []
        for c in range(self.n_classes):
            at_indice = np.where(np.array(neighs_y) == c)
            if len(at_indice) == 0:
                feats.append(999)
                continue
            print("lol", neighs_dist[at_indice[0]])
            feats.append(neighs_dist[at_indice[0]])
            # YOUR CODE GOES HERE
        
        assert len(feats) == self.n_classes
        return_list += [feats]
        
        '''
            4. Minimum *normalized* distance to objects of each class
               As 3. but we normalize (divide) the distances
               by the distance to the closest neighbor.
               
               If there are no neighboring objects of some classes, 
               Then set distance to that class to be 999.
               
               Do not forget to add self.eps to denominator.
        '''
        feats = []
        denom = neighs_dist[0] + self.eps
        for c in range(self.n_classes):
            # YOUR CODE GOES HERE
            at_indice = np.where(np.array(neighs_y) == c)
            if len(at_indice) == 0:
                feats.append(999/denom)
                continue
            feats.append(neighs_dist[at_indice[0]]/denom)
            
        assert len(feats) == self.n_classes
        return_list += [feats]
        
        '''
            5. 
               5.1 Distance to Kth neighbor
                   Think of this as of quantiles of a distribution
               5.2 Distance to Kth neighbor normalized by 
                   distance to the first neighbor
               
               feat_51, feat_52 are answers to 5.1. and 5.2.
               should be scalars
               
               Do not forget to add self.eps to denominator.
        '''
        for k in self.k_list:
            
            feat_51 = neighs_dist[k-1]# YOUR CODE GOES HERE
            feat_52 = neighs_dist[k-1]/(neighs_dist[0]+self.eps)# YOUR CODE GOES HERE
            
            return_list += [[feat_51, feat_52]]
        
        '''
            6. Mean distance to neighbors of each class for each K from `k_list` 
                   For each class select the neighbors of that class among K nearest neighbors 
                   and compute the average distance to those objects
                   
                   If there are no objects of a certain class among K neighbors, set mean distance to 999
                   
               You can use `np.bincount` with appropriate weights
               Don't forget, that if you divide by something, 
               You need to add `self.eps` to denominator.
        '''
        for k in self.k_list:
            feats = []
            for c in range(self.n_classes):
                at_indice = np.where(np.array(neighs_y) == c)
                if len(at_indice) == 0:
                    feats.append(999)
                    continue
                neig_len = len(at_indice)
                if neig_len < k:
                    feats.append(np.mean(neighs_dist[at_indice][0:neig_len]))
                else:
                    feats.append(np.mean(neighs_dist[at_indice][0:k]))
                
            
            # YOUR CODE GOES IN HERE
            
            assert len(feats) == self.n_classes
            return_list += [feats]
        
        
        # merge
        knn_feats = np.hstack(return_list)
        
        assert knn_feats.shape == (239,) or knn_feats.shape == (239, 1)
        return knn_feats

In [44]:
# a list of K in KNN, starts with one 
k_list = [3, 8, 32]

# Load correct features
true_knn_feats_first50 = np.load('knn_feats_test_first50.npy')

# Create instance of our KNN feature extractor
NNF = NearestNeighborsFeats(n_jobs=1, k_list=k_list, metric='minkowski')

# Fit on train set
NNF.fit(X, Y)

# Get features for test
test_knn_feats = NNF.predict(X_test[:50])

print((test_knn_feats - true_knn_feats_first50)[25])

print(test_knn_feats.shape)
print(true_knn_feats_first50.shape)


# This should be zero
print ('Deviation from ground thruth features: %f' % np.abs(test_knn_feats - true_knn_feats_first50).sum())

deviation =np.abs(test_knn_feats - true_knn_feats_first50).sum(0)
for m in np.where(deviation > 1e-3)[0]: 
    p = np.where(np.array([87, 88, 117, 146, 152, 239]) > m)[0][0]
    print ('There is a problem in feature %d, which is a part of section %d.' % (m, p + 1))

lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol [ 0.79725457  1.08043986  1.08403761  1.08460959  1.08460959  1.0859543
  1.09762169  1.09762169  1.11637103  1.13225786  1.13399949  1.13575065
  1.14203851  1.14483732  1.14486332  1.15042534  1.16463852  1.16957448
  1.16993127  1.17627255  1.17922265  1.18348784  1.18418232  1.18433118
  1.1925771   1.1985948   1.19872066  1.20144225  1.20746672  1.21786223
  1.21818852  1.21896896]
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol [ 1.2148284   1.28506166  1.30206464  1.31413284]
lol []
lol []
lol [ 1.30530098]
lol []
lol []
lol []
lol []
lol []
lol [ 1.21357424  1.2795603   1.28521756  1.29077389]
lol []
lol [ 1.16767458  1.22884204  1.2296545   1.24061467  1.26049973  1.26224057
  1.26663269  1.27941247  1.28763531  1.28932821  1.2959705   1.30068055
  1.3013567   1.30326929  1.30830478  1.31039502  

lol []
lol []
lol []
lol [ 1.15285759]
lol []
lol []
lol []
lol []
lol [ 0.          0.29796781  0.38334768  0.39940362  0.85890306  0.8857326
  0.8857326   0.89330504  0.93226616  0.94921937  1.09331786  1.10708345
  1.1259889   1.13369     1.13605437  1.15064777  1.15575197  1.15575197
  1.16357509  1.16899334  1.17512928  1.19776265  1.19938726  1.22120117]
lol []
lol []
lol []
lol []
lol [ 1.13263353  1.19997697  1.21778824  1.21833273]
lol []
lol [ 1.16906534  1.20264652]
lol []
lol []
lol []
lol []
lol []
lol [ 1.22610992]
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol [ 0.47728922  0.83575441  0.84862899  0.899837    0.899837    0.90563743
  0.90563743  0.91374826  0.91374826  0.94165107  0.94165107  0.94519082
  0.94532823  0.94552165  0.96193207  0.98147828  0.99835299  1.00227168
  1.00809515  1.00837668  1.01318428  1.0220613   1.0288638   1.02994427
  1.03200248  1.03272609  1.0

lol [ 1.41421356  1.41421356]
lol [ 1.084486    1.41421356]
lol []
lol []
lol [ 1.41421356  1.41421356  1.41421356]
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol [ 1.21172904]
lol [ 1.41421356]
lol [ 0.95164678  0.97465618  1.05470275  1.06647414  1.08942207  1.09086215
  1.10341436  1.1211742   1.13581184  1.41421356  1.41421356]
lol []
lol []
lol [ 1.41421356]
lol [ 1.41421356  1.41421356  1.41421356]
lol []
lol [ 1.41421356  1.41421356  1.41421356]
lol [ 1.41421356  1.41421356  1.41421356]
lol []
lol []
lol []
lol []
lol []
lol [ 1.03073935  1.41421356]
lol [ 1.18409282  1.19749711]
lol [ 1.1641427   1.18534659  1.20864023  1.21134329  1.216687  ]
lol []
lol []
lol [ 1.16821943  1.16903626  1.19541198  1.19630256  1.20667274  1.21917673]
lol []
lol [ 1.20646055]
lol []
lol []
lol []
lol []
lol []
lol []
lol [ 1.06688562  1.06688562  1.06688562  1.08858877  1.11118702  1.12453468
  1.1696917   1.2010042   1.20535936  1.20822749  1.21053509  1.21157879
  1.21226237]
lol 

lol []
lol [ 0.94544257  1.00196092  1.00549359  1.02669764  1.03061999  1.03372536
  1.12264079  1.13685115  1.14458415  1.15773862  1.16200789  1.16532789
  1.17060949  1.17245298  1.18174246  1.18174246  1.18361894  1.18408713
  1.19123068  1.19321303  1.19387866  1.19402604  1.19535324  1.20028004
  1.20396553  1.20659356  1.2093161   1.20968423]
lol []
lol []
lol []
lol []
lol [ 1.14088225]
lol []
lol []
lol []
lol []
lol [ 1.11497094]
lol []
lol [ 1.20767673]
lol [ 1.17186153]
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol [ 1.09961128  1.13325765  1.14308368  1.15487825  1.18983915  1.19905725
  1.2058954   1.2058954   1.2058954   1.22752463  1.23328277  1.23381846
  1.23949486  1.26092693  1.26508997  1.26945334  1.27330053  1.27498221
  1.28299449  1.28816478]
lol [ 1.2502036   1.26939816]
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol []
lol [ 1.23141097  1.24063425  1.27733783  1.

ValueError: operands could not be broadcast together with shapes (0,) (32,) 

In [ ]:
for metric in ['minkowski', 'cosine']:
    print (metric)
    
    # Create instance of our KNN feature extractor
    NNF = NearestNeighborsFeats(n_jobs=4, k_list=k_list, metric=metric)
    
    # Fit on train set
    NNF.fit(X, Y)

    # Get features for test
    test_knn_feats = NNF.predict(X_test)
    
    # Dump the features to disk
    np.save('data/knn_feats_%s_test.npy' % metric , test_knn_feats)

In [ ]:
# Differently from other homework we will not implement OOF predictions ourselves
# but use sklearn's `cross_val_predict`
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold

# We will use two metrics for KNN
for metric in ['minkowski', 'cosine']:
    print (metric)
    
    # Set up splitting scheme, use StratifiedKFold
    # use skf_seed and n_splits defined above with shuffle=True
    skf = # YOUR CODE GOES HERE
    
    # Create instance of our KNN feature extractor
    # n_jobs can be larger than the number of cores
    NNF = NearestNeighborsFeats(n_jobs=4, k_list=k_list, metric=metric)
    
    # Get KNN features using OOF use cross_val_predict with right parameters
    preds = # YOUR CODE GOES HERE
    
    # Save the features
    np.save('data/knn_feats_%s_train.npy' % metric, preds)

In [ ]:
s = 0
for metric in ['minkowski', 'cosine']:
    knn_feats_train = np.load('data/knn_feats_%s_train.npy' % metric)
    knn_feats_test = np.load('data/knn_feats_%s_test.npy' % metric)
    
    s += knn_feats_train.mean() + knn_feats_test.mean()
    
answer = np.floor(s)
print (answer)